In [1]:
from transformers import BertModel,BertTokenizer,BertConfig

In [132]:
from transformers import RobertaTokenizer,RobertaModel,BertConfig

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader

In [148]:
from sklearn.metrics import accuracy_score

In [3]:
import json

In [125]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [123]:
device

device(type='cuda')

In [138]:
config = BertConfig.from_pretrained('D:\BDT\MSBD5018NLP\Sarcasm Detection\\bert\\config.json')
bert = BertModel.from_pretrained('D:\BDT\MSBD5018NLP\Sarcasm Detection\\bert\\pytorch_model.bin',config=config)

Some weights of the model checkpoint at D:\BDT\MSBD5018NLP\Sarcasm Detection\bert\pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [134]:
tokenizer = BertTokenizer('D:\BDT\MSBD5018NLP\Sarcasm Detection\\bert\\vocab.txt')

In [183]:
data = []
with open('D:\BDT\MSBD5018NLP\Sarcasm Detection\Sarcasm_Headlines_Dataset_v2.json\Sarcasm_Headlines_Dataset_v2.json', 'r',encoding='utf-8') as j:
    max = 0
    for i in j:
        item = json.loads(i)
        headline = item['headline']
        if len(headline)>max:
            max = len(headline)
        label = int(item['is_sarcastic'])
        data.append((headline,label))
    print(max)

926


In [182]:
data

[('thirtysomething scientists unveil doomsday clock of hair loss', 1),
 ('dem rep. totally nails why congress is falling short on gender, racial equality',
  0),
 ('eat your veggies: 9 deliciously different recipes', 0),
 ('inclement weather prevents liar from getting to work', 1),
 ("mother comes pretty close to using word 'streaming' correctly", 1),
 ('my white inheritance', 0),
 ('5 ways to file your taxes with less stress', 0),
 ("richard branson's global-warming donation nearly as much as cost of failed balloon trips",
  1),
 ('shadow government getting too large to meet in marriott conference room b',
  1),
 ('lots of parents know this scenario', 0),
 ('this lesbian is considered a father in indiana (and an amazing one at that)',
  0),
 ("amanda peet told her daughter sex is 'a special hug'", 0),
 ('what to know regarding current treatments for ebola', 0),
 ("chris christie suggests hillary clinton was to blame for boko haram's kidnapping of hundreds of schoolgirls",
  0),
 ('for

In [191]:
class DS(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        input = tokenizer(self.data[idx][0],max_length = 128,padding='max_length',truncation = True)
        label = self.data[idx][1]
        return torch.tensor(input['input_ids']).to(device),torch.tensor(input['attention_mask']).to(device),\
                torch.tensor(input['token_type_ids']).to(device),torch.tensor(label).to(device)

In [155]:
class IntraAttention(nn.Module):
    def __init__(self,embedding_dim=768,hidden_dim=128):
        super().__init__()
        self.T1 = nn.Linear(embedding_dim*2,hidden_dim,bias = True)
        self.relu = torch.nn.ReLU()
        self.T2 = nn.Linear(hidden_dim,1,bias = True)
    
    def forward(self,embeddings):
        #print('embeddings',embeddings.size())
        m = embeddings.size()[1]
        #x = embeddings.squeeze(0)
        x = embeddings
        #print('x',x.size())
        pairs = torch.cat([x.unsqueeze(2).expand(-1,-1,m,-1),x.unsqueeze(1).expand(-1,m,-1,-1)],dim=3)
        #print('pair',pairs.size())
        x = self.T1(pairs)
        x = self.relu(x)
        #print('linear',x.size())
        x = self.T2(x).squeeze(3)
        #print('attention',x.size())
        
        return x
        

In [165]:
class MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = bert
        for param in self.bert.parameters():
            param.requires_grad = False
        self.attention = IntraAttention()
        self.softmax = nn.Softmax(dim=-1)
        self.lstm = nn.LSTM(768,128,batch_first = True)
        self.l = nn.Linear(768+128,300)
        self.relu = torch.nn.ReLU()
        self.cls = nn.Linear(300,2,bias=True)
    
    def forward(self,input_ids,attention_mask,token_type_ids):
        embedding = self.bert(input_ids,attention_mask,token_type_ids)[0]#(1,512,768)
        max_len = embedding.size()[1]
        batch_size = embedding.size()[0]
        #embedding = embedding*attention_mask.unsqueeze(2).expand(-1,-1,768)
        #print(embedding)
        
        attention = self.attention(embedding)
        mask = attention_mask.unsqueeze(1).expand(-1,max_len,-1)
        mask = torch.transpose(mask,1,2)*mask
        mask = mask - torch.eye(max_len).repeat(batch_size,1,1).to(device)
        
        mask = mask.bool()
        attention = attention.masked_fill(mask == 0, -1e9)
        #print(attention)
        p_attn = self.softmax(attention)
        #print(p_attn.shape)
        row_max,_ = torch.max(p_attn,dim=-1,keepdim=True)
        #print(row_max.shape,attention_mask.shape)
        a = row_max.squeeze(-1)*attention_mask
        pair_embedding = torch.matmul(a.unsqueeze(1),embedding).squeeze(1)
        
        context,_ = self.lstm(embedding)
        x = torch.cat([pair_embedding,context[:,-1,:]],dim=1)
        x = self.l(x)
        x = self.relu(x)
        x = self.cls(x)
        x = self.softmax(x)
        #torch.matmul(p_attn, value)
        #attention = attention[mask]
        return x

In [195]:
ratio = 0.8
index = int(len(data)*ratio)
trainset = DS(data[:index])
testset = DS(data[index:])

In [158]:
len(testset)

5724

In [196]:
trainloader = DataLoader(dataset=trainset, batch_size=4)
testloader = DataLoader(dataset=testset, batch_size=4)

In [177]:
model = MODEL()

In [200]:
#model.load_state_dict(torch.load(PATH))

In [179]:
model = model.to(device)

In [180]:
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-4)

In [149]:
len(d)

28619

In [107]:
loss_function = nn.CrossEntropyLoss()

In [181]:
epoch_loss = 0.0
model.train()
true = []
pred =[]
best_acc = 0.0
for idx,x in enumerate(trainloader):
    model.train()
    #print(x[-1])
    optimizer.zero_grad()
    output = model(x[0],x[1],x[2])
    pred.extend(output.argmax(dim=1).tolist())
    true.extend(x[-1].tolist())
    loss = loss_function(output,x[-1])
    epoch_loss+=loss.item()
    loss.backward()
    optimizer.step()
    
    if (idx+1)%20==0:
        tpred = []
        ttrue = []
        tt = 0.0
        model.eval()
        count = 0
        for tidx,tx in enumerate(testloader):
            count+=1
            toutput = model(tx[0],tx[1],tx[2])
            tloss = loss_function(toutput,tx[-1])
            tt+=tloss.item()
            tpred.extend(toutput.argmax(dim=1).tolist())
            ttrue.extend(tx[-1].tolist())
        acc = accuracy_score(ttrue,tpred)
        print('#'*10+'batch id: ',idx,'accuracy: ',acc,'loss: ',tt/count)
        if acc>best_acc:
            torch.save(model.state_dict(), PATH)
            best_acc = acc
            print('-'*10+'batch id: ',idx,'save model!')
            

##########batch id:  19 accuracy:  0.5267295597484277 loss:  0.6765922414181867
##########batch id:  39 accuracy:  0.6324248777078966 loss:  0.6304465638516584
##########batch id:  59 accuracy:  0.7398672257162823 loss:  0.5637958415066088
##########batch id:  79 accuracy:  0.7562893081761006 loss:  0.5462768616416419
##########batch id:  99 accuracy:  0.76659678546471 loss:  0.5373478499973345
##########batch id:  119 accuracy:  0.7921034241788959 loss:  0.5179619162046017
##########batch id:  139 accuracy:  0.7851153039832285 loss:  0.5233395215225419
##########batch id:  159 accuracy:  0.7945492662473794 loss:  0.5128233411735292
##########batch id:  179 accuracy:  0.7723619846261356 loss:  0.5308973092994517
##########batch id:  199 accuracy:  0.7969951083158631 loss:  0.5089283928139738
##########batch id:  219 accuracy:  0.7894828791055206 loss:  0.5167401603766041
##########batch id:  239 accuracy:  0.796470999301188 loss:  0.5094119685667532
##########batch id:  259 accuracy:  

RuntimeError: stack expects each tensor to be equal size, but got [128] at entry 0 and [193] at entry 2

In [197]:
epoch = 1
for epoch_id in range(epoch):
    epoch_loss = 0.0
    model.train()
    true = []
    pred =[]
    for idx,x in enumerate(trainloader):
        #if idx<1820:continue
        model.train()
        #print(x[-1])
        optimizer.zero_grad()
        output = model(x[0],x[1],x[2])
        pred.extend(output.argmax(dim=1).tolist())
        true.extend(x[-1].tolist())
        loss = loss_function(output,x[-1])
        epoch_loss+=loss.item()
        loss.backward()
        optimizer.step()
    
        if (idx+1)%20==0:
            tpred = []
            ttrue = []
            tt = 0.0
            model.eval()
            count = 0
            for tidx,tx in enumerate(testloader):
                count+=1
                toutput = model(tx[0],tx[1],tx[2])
                tloss = loss_function(toutput,tx[-1])
                tt+=tloss.item()
                tpred.extend(toutput.argmax(dim=1).tolist())
                ttrue.extend(tx[-1].tolist())
            print('#'*10+'batch id: ',idx,'accuracy: ',accuracy_score(ttrue,tpred),'loss: ',tt/count)

##########batch id:  1839 accuracy:  0.8432914046121593 loss:  0.46346828339401447
##########batch id:  1859 accuracy:  0.8404961565338924 loss:  0.4644733831276017
##########batch id:  1879 accuracy:  0.8394479385045422 loss:  0.46616710198666983
##########batch id:  1899 accuracy:  0.8378756114605171 loss:  0.466734562363348
##########batch id:  1919 accuracy:  0.8186582809224319 loss:  0.4847376408442011
##########batch id:  1939 accuracy:  0.8022361984626135 loss:  0.4981184056345022
##########batch id:  1959 accuracy:  0.8204053109713487 loss:  0.48394738376182245
##########batch id:  1979 accuracy:  0.8459119496855346 loss:  0.46129461648079334
##########batch id:  1999 accuracy:  0.8242487770789657 loss:  0.47764725286505755
##########batch id:  2019 accuracy:  0.8380503144654088 loss:  0.4672462952711797
##########batch id:  2039 accuracy:  0.8377009084556254 loss:  0.46633319370318427
##########batch id:  2059 accuracy:  0.8310621942697415 loss:  0.4740180432963088
##########b

##########batch id:  3839 accuracy:  0.8572676450034941 loss:  0.44887395243724687
##########batch id:  3859 accuracy:  0.8520265548567435 loss:  0.45516181716878956
##########batch id:  3879 accuracy:  0.8581411600279525 loss:  0.4475343984211349
##########batch id:  3899 accuracy:  0.860587002096436 loss:  0.44665728085612516
##########batch id:  3919 accuracy:  0.8590146750524109 loss:  0.4502669463700802
##########batch id:  3939 accuracy:  0.8581411600279525 loss:  0.44628563256716913
##########batch id:  3959 accuracy:  0.8630328441649197 loss:  0.44449438649320167
##########batch id:  3979 accuracy:  0.8602375960866527 loss:  0.4448738358472295
##########batch id:  3999 accuracy:  0.8523759608665269 loss:  0.4529683074486331
##########batch id:  4019 accuracy:  0.8120195667365478 loss:  0.49176817253533983
##########batch id:  4039 accuracy:  0.8296645702306079 loss:  0.4758643879913767
##########batch id:  4059 accuracy:  0.8446890286512928 loss:  0.4629044932431062
##########b

In [199]:
tpred = []
ttrue = []
model.eval()
for tidx,tx in enumerate(trainloader):
    if (tidx+1)%20==0:
        print(tidx)
    count+=1
    toutput = model(tx[0],tx[1],tx[2])
    tloss = loss_function(toutput,tx[-1])
    tt+=tloss.item()
    tpred.extend(toutput.argmax(dim=1).tolist())
    ttrue.extend(tx[-1].tolist())
print('#'*10+'batch id: ',idx,'accuracy: ',accuracy_score(ttrue,tpred),'loss: ',tt/count)

19
39
59
79
99
119
139
159
179
199
219
239
259
279
299
319
339
359
379
399
419
439
459
479
499
519
539
559
579
599
619
639
659
679
699
719
739
759
779
799
819
839
859
879
899
919
939
959
979
999
1019
1039
1059
1079
1099
1119
1139
1159
1179
1199
1219
1239
1259
1279
1299
1319
1339
1359
1379
1399
1419
1439
1459
1479
1499
1519
1539
1559
1579
1599
1619
1639
1659
1679
1699
1719
1739
1759
1779
1799
1819
1839
1859
1879
1899
1919
1939
1959
1979
1999
2019
2039
2059
2079
2099
2119
2139
2159
2179
2199
2219
2239
2259
2279
2299
2319
2339
2359
2379
2399
2419
2439
2459
2479
2499
2519
2539
2559
2579
2599
2619
2639
2659
2679
2699
2719
2739
2759
2779
2799
2819
2839
2859
2879
2899
2919
2939
2959
2979
2999
3019
3039
3059
3079
3099
3119
3139
3159
3179
3199
3219
3239
3259
3279
3299
3319
3339
3359
3379
3399
3419
3439
3459
3479
3499
3519
3539
3559
3579
3599
3619
3639
3659
3679
3699
3719
3739
3759
3779
3799
3819
3839
3859
3879
3899
3919
3939
3959
3979
3999
4019
4039
4059
4079
4099
4119
4139
4159
4179
4199
4219


In [ ]:
accuracy_score(true,pred)

In [198]:
PATH = 'D:\BDT\MSBD5018NLP\Sarcasm Detection\\bert\\save\\weights'
torch.save(model.state_dict(), PATH)

In [193]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [114]:
output

tensor([[0.3863, 0.6137],
        [0.4239, 0.5761],
        [0.4087, 0.5913],
        [0.4127, 0.5873]], grad_fn=<SoftmaxBackward>)

In [136]:
model(torch.tensor([input['input_ids']]),torch.tensor([input['attention_mask']]),torch.tensor([input['token_type_ids']]))[0].shape

embeddings torch.Size([1, 512, 768])
x torch.Size([1, 512, 768])
pair torch.Size([1, 512, 512, 1536])
linear torch.Size([1, 512, 512, 128])
attention torch.Size([1, 512, 512])
torch.Size([1, 512, 1]) torch.Size([1, 512])


torch.Size([2])